<a href="https://colab.research.google.com/github/nikewinchester/Text-Generator-LSTM/blob/main/word-level-text-generator-bilstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np
import re
import tensorflow as tf

In [2]:
import requests
from bs4 import BeautifulSoup
gitfile = requests.get('https://raw.githubusercontent.com/nikewinchester/Text-Generator-LSTM/main/NLP task.txt')
data = str(BeautifulSoup(gitfile.content, "html.parser"))

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
tokenizer = Tokenizer()
corpus = data.lower().split(".")
for i in range(0,len(corpus)):
  s = re.sub(' +',' ',(re.sub(r'[^\w]', ' ', corpus[i])))
  corpus[i] = s
print(len(corpus))
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

6417


In [4]:
input_sequences = []
for j in corpus:
 token_list = tokenizer.texts_to_sequences([j])[0]
 for i in range(1, len(token_list)):
  n_gram_sequence = token_list[:i+1]
  input_sequences.append(n_gram_sequence)

In [5]:
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [6]:
# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = ku.to_categorical(label, num_classes=total_words)

In [9]:
from tensorflow import keras
model = keras.models.load_model('/content/gdrive/MyDrive/Text-Generator/final_model.hdf5')

In [14]:
seed_text = "how are you"
next_words = 5
  
for _ in range(next_words):
 token_list = tokenizer.texts_to_sequences([seed_text])[0]
 token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
 predicted = np.argmax(model.predict(token_list), axis=-1)
 output_word = ""
 for word, index in tokenizer.word_index.items():
  if index == predicted:
   output_word = word
   break
 seed_text += " " + output_word
print(seed_text)

how are you getting off for some years


In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
from keras.callbacks import ModelCheckpoint

filepath = "model_training.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')
callbacks = [checkpoint]

In [ ]:
history = model.fit(predictors, label, epochs=500, verbose=1,callbacks=callbacks)

In [ ]:
model.save("model1.h5")